<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Get-TPU-device" data-toc-modified-id="Get-TPU-device-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Get TPU device</a></span></li></ul></div>

In [1]:
# default_exp core

# Core XLA extensions

> API details.

In [2]:
#hide
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#hide
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [4]:
#hide
!curl https://course.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   322  100   322    0     0   1490      0 --:--:-- --:--:-- --:--:--  1490
Updating fastai...
Done.


In [5]:
#hide
VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4139  100  4139    0     0  51098      0 --:--:-- --:--:-- --:--:-- 50475
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200325 ...
     |████████████████████████████████| 61kB 3.0MB/s 
Uninstalling torch-1.5.1+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.5.1+cu101
Uninstalling torchvision-0.6.1+cu101:
  Successfully uninstalled torchvision-0.6.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
\ [1 files][ 83.4 MiB/ 83.4 MiB]                                                
Operation completed over 1 objects/83.4 MiB.                             

In [6]:
#hide
!pip install fastai2 --upgrade

     |████████████████████████████████| 194kB 3.4MB/s 


In [7]:
#hide
%cd /content/drive/My\ Drive/course-v4/

/content/drive/My Drive/course-v4


In [8]:
#hide
!pip install -r requirements.txt

     |████████████████████████████████| 51kB 2.0MB/s 
     |████████████████████████████████| 51kB 3.7MB/s 
     |████████████████████████████████| 1.1MB 4.5MB/s 
     |████████████████████████████████| 40kB 5.1MB/s 
     |████████████████████████████████| 92kB 7.6MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 2.7MB 10.8MB/s 


In [9]:
#hide
%cd nbs

/content/drive/My Drive/course-v4/nbs


In [10]:
#hide
from utils import *

In [11]:
#hide
from nbdev.showdoc import *

In [12]:
#hide
%cd /content/drive/My\ Drive/fastai_xla_extensions

/content/drive/My Drive/fastai_xla_extensions


In [13]:
#exporti
from fastai2.optimizer import *

In [14]:
#exporti
import torch_xla.core.xla_model as xm

In [15]:
#export
class XLAOptFuncWrapper:
    def __init__(self, f):
        self.f = f 
    def __call__(self, *args, **kwargs):
        opt = self.f(*args, **kwargs)
        optim_wrapper = OptimWrapper(opt)
        def xla_step():
            xm.optimizer_step(opt, barrier=True)
        # monkeypatch optim_wrapper with xla_step
        optim_wrapper.step = xla_step
        return optim_wrapper
    

In [16]:
from fastai2.vision.all import *

In [17]:
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path

In [18]:
datablock = DataBlock(
    blocks=(ImageBlock(cls=PILImageBW),CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter=GrandparentSplitter(),
    item_tfms=Resize(28),
    batch_tfms=[]
)

## Get TPU device

In [19]:
tpu_dev = xm.xla_device()

In [20]:
dls = datablock.dataloaders(path, device=tpu_dev)

In [21]:
XLAAdam = XLAOptFuncWrapper(Adam)

In [22]:
learner = cnn_learner(dls, resnet18, metrics=accuracy, opt_func=XLAAdam)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [23]:
learner.fine_tune(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.118350,0.047767,0.987733,00:32


epoch,train_loss,valid_loss,accuracy,time
0,0.027038,0.002208,0.999509,00:29


In [24]:
!nbdev_build_docs

converting: /content/drive/My Drive/fastai_xla_extensions/nbs/00_core.ipynb
converting: /content/drive/My Drive/fastai_xla_extensions/nbs/index.ipynb
converting /content/drive/My Drive/fastai_xla_extensions/nbs/index.ipynb to README.md
